In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import datetime
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from docx import Document
from docx.shared import Inches

In [2]:
from matplotlib.font_manager import FontProperties
from pathlib import Path
# point to the font location with an absolute path
nirm = Path('c:/Windows/Fonts/kartika.ttf')

# configure the  font
mal_font = FontProperties(fname=nirm)

In [17]:
mal = np.load('data/mal_chars_126.npy')

In [40]:
def predict_word(ch):
    pred = ''
    lis = ch.split(' ')
    char_list = [chr(int(i)) for i in lis]
    pred += ''.join(char_list)
    return pred

In [4]:
BATCH_SIZE = 32
NORMAL_EPOCHS= 6
FINETUNE_EPOCHS = 8
img_height = 224
img_width = 224

In [5]:
data_dir = 'data/dataset/'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=12,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

Found 31476 files belonging to 126 classes.
Using 22034 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=12,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

Found 31476 files belonging to 126 classes.
Using 9442 files for validation.


In [7]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

In [8]:
class_names = train_ds.class_names


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
NUM_CLASSES = len(class_names)
IMG_SHAPE = (img_width, img_height, 3)

In [10]:
import tensorflow_datasets as tfds
def dataset_to_numpy(ds):
    """
    Convert tensorflow dataset to numpy arrays
    """
    images_batch = []
    labels_batch = []
    images = []
    labels = []
    

    # Iterate over a dataset
    for i, (image, label) in enumerate(tfds.as_numpy(ds)):
        images_batch.append(image)
        labels_batch.append(label)

    for i, img_batch in enumerate(images_batch):
        if i < 3:
            print(img_batch.shape, labels_batch[i])
        for j,img in enumerate(img_batch):
            images.append(img)
            labels.append(labels_batch[i][j])

    return images, labels

In [11]:
y_test, y_true  = dataset_to_numpy(test_dataset)
y_test = np.array(y_test)


(32, 224, 224, 3) [ 64  93   6  42  42   0  21 107 107  27  78  36  49  77  39  69  16  45
  20  39  18 108  47  98 111 102 103  68 115 122  74  78]
(32, 224, 224, 3) [ 25 122  21  33  58  83 103  88  38  70  62  80  19  83 122  79   9  50
  36  45  87  88   2 120  74  36 116 101  92  98  63 111]
(32, 224, 224, 3) [ 90  36 102  84  57 112  59  62 123  65 115 118  86  88  27  70  81  29
  67  95 110  39   6   6  92   7  60  54  88  24 116  61]


In [12]:
def generate_doc(model_folder,name,config):
    document = Document()
    document.add_heading(f'Model {name}: ', 0)
    document.add_heading(f'Batch Size: {config["batch"]}', 5)
    document.add_heading(f'Normal epoch: {config["normal_epoch"]}', 5)
    document.add_heading(f'Finetuned epoch: {config["finetuned_epoch"]}', 5)
    document.add_heading('Accuracy', 3)
    document.add_picture(model_folder + '\\accuracy.png', width=Inches(6))
    document.add_heading('Loss', 3)
    document.add_picture(model_folder + '\\loss.png', width=Inches(6))
    document.add_heading('Loss', 3)
    document.add_picture(model_folder + '\\prediction.png', width=Inches(6))
    report = config['report']
    table = document.add_table(rows=1,cols=len(report.keys()))
    table.style = 'TableGrid'
    # Adding heading in the 1st row of the table
    row = table.rows[0].cells
    for i, key in enumerate(list(report.keys())):
        row[i].text = key
    for index, row_data in report.iterrows():
        row = table.add_row().cells
        for j, key in enumerate(list(report.keys())):
            if isinstance(row_data[key],float):
                row[j].text = f"{row_data[key]:.2f}"
            else:
                row[j].text = f"{row_data[key]}"
    file_name = model_folder+ f"\\report.docx"
    document.save(file_name)

In [13]:
crd = tf.data.experimental.cardinality(train_ds)

In [14]:
crd

<tf.Tensor: shape=(), dtype=int64, numpy=689>

In [18]:
data, label  = dataset_to_numpy(train_ds)
data = np.array(data)

(32, 224, 224, 3) [120  57  52  53  38  85  22 125  15  21  94  28  71  48  19  32 110  99
  65  13  66  67  33  30  54  65 100  83   7  71   7  91]
(32, 224, 224, 3) [ 44  97  35  86 107  57  39  80 108  88  39  75  68  69  96  94  10  79
 110  12 125  59  28  23  68  16  91 122   7  99   9  51]
(32, 224, 224, 3) [  6  97  69 125  22   1 122  96  24 119 123  87 124 109 106  74 100 108
   9  30  59  62 101 125 105 113  94 120 125  80 121  73]


In [58]:
char_data = {}
ds_len = len(label)
chars = list(map(lambda x: predict_word(x),mal))
char_data = dict.fromkeys(chars, 0)
for i in range(ds_len):
    char_name = chars[label[i]]
    if char_name in char_data.keys():
        char_data[char_name] += 1
    else:
        char_data[char_name] = 1
        
    
    

In [60]:
chars_df = pd.DataFrame([(k, v) for k, v in char_data.items()],
                       columns=['character', 'number of images'])

In [59]:
char_data

{'ഁ': 165,
 'ം': 178,
 'ഃ': 175,
 'അ': 169,
 'ആ': 174,
 'ഇ': 175,
 'ഉ': 172,
 'ഋ': 169,
 'എ': 193,
 'ഏ': 177,
 'ഒ': 179,
 'ക': 170,
 'ക്ക': 185,
 'ക്ത': 176,
 'ക്ര': 173,
 'ക്ല': 169,
 'ക്ഷ': 195,
 'ഖ': 171,
 'ഗ': 173,
 'ഗ്ഗ': 168,
 'ഗ്ന': 169,
 'ഗ്മ': 175,
 'ഗ്ര': 172,
 'ഗ്ല': 176,
 'ഘ': 175,
 'ങ': 179,
 'ങ്ക': 172,
 'ങ്ങ': 167,
 'ച': 167,
 'ച്ച': 168,
 'ച്ഛ': 190,
 'ഛ': 171,
 'ജ': 185,
 'ജ്ജ': 158,
 'ജ്ഞ': 160,
 'ഝ': 172,
 'ഞ': 164,
 'ഞ്ച': 167,
 'ഞ്ഞ': 176,
 'ട': 177,
 'ട്ട': 186,
 'ഠ': 182,
 'ഡ': 193,
 'ഡ്ഡ': 179,
 'ഢ': 174,
 'ണ': 165,
 'ണ്ട': 183,
 'ണ്ഡ': 158,
 'ണ്ണ': 176,
 'ണ്മ': 183,
 'ത': 170,
 'ത്ത': 184,
 'ത്ഥ': 181,
 'ത്ഭ': 183,
 'ത്മ': 175,
 'ത്ര': 172,
 'ത്സ': 182,
 'ഥ': 181,
 'ദ': 166,
 'ദ്ദ': 180,
 'ദ്ധ': 172,
 'ദ്ര': 175,
 'ധ': 176,
 'ന': 180,
 'ന്ത': 163,
 'ന്ഥ': 178,
 'ന്ദ': 192,
 'ന്ധ': 176,
 'ന്ന': 164,
 'പ': 176,
 'പ്പ': 184,
 'പ്ല': 174,
 'ഫ': 171,
 'ബ': 163,
 'ബ്ദ': 154,
 'ബ്ധ': 176,
 'ബ്ബ': 175,
 'ബ്ല': 163,
 'ഭ': 167,
 'മ': 182,
 'മ്പ': 179,
 'മ്മ': 171,
 'മ്ല'

In [61]:
chars_df.head()

,character,number of images
0,ഁ,165
1,ം,178
2,ഃ,175
3,അ,169
4,ആ,174


In [67]:
chars_df.to_csv('./results/train_data.csv')
chars_df.to_html('./results/train_data.html')
chars_df.to_excel('./results/train_data.xlsx')